In [1]:
!nvidia-smi

Tue Jul  2 17:32:13 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.78                 Driver Version: 551.78         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce MX350         WDDM  |   00000000:02:00.0 Off |                  N/A |
| N/A   41C    P0             N/A / ERR!  |       0MiB /   2048MiB |      2%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
from torch import nn
torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [3]:
torch.cuda.device_count()

1

定义了两个方便的函数，这两个函数允许我们在不存在所需所有GPU的情况下运行代码。

In [4]:
def try_gpu(i=0): 
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus(): 
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()


(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

查询张量所在的设备

In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

存储在gpu上

In [6]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

在第二个GPU上创建一个随机张量。

In [8]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y
#只有一个GPU 会返回到CPU上面 后面的代码略过

tensor([[0.2190, 0.0310, 0.5369],
        [0.5427, 0.4866, 0.5616]])

神经网络和GPU

In [10]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

net(X)

tensor([[-0.4904],
        [-0.4904]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [11]:
#让我们确认模型参数存储在同一个GPU上。
net[0].weight.data.device

device(type='cuda', index=0)